In [2]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import skimage
import os
import torch.nn as nn
import torch.nn.functional as nnF
from torch.utils.data import DataLoader as torch_dataloader
from torch.utils.data import Dataset as torch_dataset
import torch.optim as optim
#from data_loader import get_dataloader
from torchvision import models
import pandas as pd
import skimage.io as io
import skimage

In [3]:
current_directory = os.getcwd()

# Define the relative path to the text file or data file
file_name = 'val.csv'  # Replace 'your_text_file.txt' with the actual file name

# Construct the absolute file path by joining the current directory and the file name
absolute_file_path = os.path.join(current_directory, file_name)

print(absolute_file_path)

/Users/yemiakj10/Desktop/MSDS 2/Machine Learning/Main Story/GradCam/val.csv


In [4]:
class RandomResNet18(nn.Module):
    def __init__(self, num_classes=2):
        super(RandomResNet18, self).__init__()
        # Load the pre-trained ResNet-18 model
        self.resnet = models.resnet18(pretrained=False)
     
       # Modify the final fully connected layer for binary classification
        num_ftrs = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_ftrs, num_classes)

    def forward(self, x):
        # Forward pass through the modified ResNet-18 model
        x = self.resnet(x)
        return x

In [5]:
class PretrainedResNet18(nn.Module):
    def __init__(self, num_classes=2):
        super(PretrainedResNet18, self).__init__()
        # Load the pre-trained ResNet-18 model
        resnet = models.resnet18(pretrained=True)
        
        # Remove the original fully connected layer
        self.features = nn.Sequential(*list(resnet.children())[:-1])
        
        # Add a new fully connected layer for binary classification
        self.fc = nn.Linear(resnet.fc.in_features, num_classes)

    def forward(self, x):
        # Forward pass through the modified ResNet-18 model
        x = self.features(x)
        # Determine the size of the feature maps
        feature_size = x.size(1)
        # Flatten the feature maps
        x = x.view(x.size(0), feature_size)
        x = self.fc(x)
        return x

In [6]:
def save_checkpoint(filename, model, optimizer, result, epoch):
    torch.save({'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'result':result},
               filename)
    print('saved:', filename)

In [7]:
class MyDataset(torch_dataset):
    def __init__(self, path, filenamelist, labellist):
        self.path=path
        self.filenamelist=filenamelist
        self.labellist=labellist
    def __len__(self):
        #return the number of data points
        return len(self.filenamelist)
    def __getitem__(self, idx):
        I=io.imread(self.path+self.filenamelist[idx])
        I=skimage.util.img_as_float32(I)
        I = I.reshape(1,I.shape[0],I.shape[1])
        I = torch.tensor(I, dtype=torch.float32)
        I = I.expand(3, I.shape[1],I.shape[2])
        label=torch.tensor(self.labellist[idx], dtype=torch.int64)
        return I, label
#%%
def get_dataloader(path='/Users/yemiakj10/Desktop/MSDS 2/Machine Learning/Main Story/GradCam/'):
    df_train = pd.read_csv(os.path.join(path, 'train.csv'))
    df_val = pd.read_csv(os.path.join(path, 'val.csv'))
    df_test = pd.read_csv(os.path.join(path, 'test.csv'))
    dataset_train = MyDataset(path, df_train['filename'].values, df_train['label'].values)
    dataset_val = MyDataset(path, df_val['filename'].values, df_val['label'].values)
    dataset_test = MyDataset(path, df_test['filename'].values, df_test['label'].values)
    loader_train = torch_dataloader(dataset_train, batch_size=32, num_workers=0,
                                    shuffle=True, pin_memory=True)
    loader_val = torch_dataloader(dataset_val, batch_size=32, num_workers=0,
                                  shuffle=False, pin_memory=True)
    loader_test = torch_dataloader(dataset_test, batch_size=32, num_workers=0,
                                   shuffle=False, pin_memory=True)

	#modify this function to return loader_train, loader_val, and loader_test
    return loader_train, loader_val, loader_test

In [8]:
def cal_accuracy(confusion):
    #input: confusion is the confusion matrix
    #output: acc is the standard classification accuracy
    M=confusion.copy().astype('float32')
    acc = M.diagonal().sum()/M.sum()    
    sens=np.zeros(M.shape[0])
    prec=np.zeros(M.shape[0]) 
    for n in range(0, M.shape[0]):
        TP=M[n,n]
        FN=np.sum(M[n,:])-TP
        FP=np.sum(M[:,n])-TP
        sens[n]=TP/(TP+FN)
        prec[n]=TP/(TP+FP)       
    return acc, sens, prec

In [9]:
def train(model, device, optimizer, dataloader, epoch):    
    model.train()#set model to training mode
    loss_train=0
    acc_train =0 
    sample_count=0
    for batch_idx, (X, Y) in enumerate(dataloader):
        X, Y = X.to(device), Y.to(device)
        optimizer.zero_grad()#clear grad of each parameter
        Z = model(X)#forward pass
        loss = nnF.cross_entropy(Z, Y)
        loss.backward()#backward pass
        optimizer.step()#update parameters
        loss_train+=loss.item()
        #do not need softmax
        Yp = Z.data.max(dim=1)[1]  # get the index of the max               
        acc_train+= torch.sum(Yp==Y).item()
        sample_count+=X.size(0)
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{:.0f}%]\tLoss: {:.6f}'.format(
                    epoch, 100. * batch_idx / len(dataloader), loss.item()))
    loss_train/=len(dataloader)
    #due to upsampling, len(dataloader.dataset) != sample_count
    #acc_train/=len(dataloader.dataset) 
    acc_train/=sample_count    
    return loss_train, acc_train

In [10]:
def test(model, device, dataloader):
    model.eval()#set model to evaluation mode
    acc_test =0
    confusion=np.zeros((5,5))
    with torch.no_grad(): # tell Pytorch not to build graph in the with section
        for batch_idx, (X, Y) in enumerate(dataloader):
            X, Y = X.to(device), Y.to(device)
            Z = model(X)#forward pass
            #do not need softmax
            Yp = Z.data.max(dim=1)[1]  # get the index of the max 
            acc_test+= torch.sum(Yp==Y).item()
            for i in range(0, 5):
                for j in range(0, 5):
                    confusion[i,j]+=torch.sum((Y==i)&(Yp==j)).item()
    acc, sens, prec=cal_accuracy(confusion)
    return acc, (confusion, sens, prec)

In [11]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_Ran = RandomResNet18()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adamax(model_Ran.parameters(), lr=0.001, weight_decay=1e-4)

/Users/yemiakj10/Anaconda/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/yemiakj10/Anaconda/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [12]:
loader_train, loader_val, loader_test = get_dataloader()
loss_train_list=[]
acc_train_list=[]
acc_val_list=[]
epoch_save=-1

In [13]:
for epoch in range(epoch_save+1, 6): #change 100 to a larger number if necessary
    #-------- training --------------------------------
    loss_train, acc_train =train(model_Ran, device, optimizer, loader_train, epoch)    
    loss_train_list.append(loss_train)
    acc_train_list.append(acc_train)
    print('epoch', epoch, 'training loss:', loss_train, 'acc:', acc_train)
    #-------- validation --------------------------------
    acc_val, other_val = test(model_Ran, device, loader_val)
    acc_val_list.append(acc_val)
    print('epoch', epoch, 'validation acc:', acc_val)
    #-------- test --------------------------------------
    #acc_test, other_test = test(model_Ran, device, loader_test)
    #acc_test_list.append(acc_test)
    #print('epoch', epoch, 'test acc:', acc_test)
    #--------save model-------------------------
    result = (loss_train_list, acc_train_list, 
              acc_val_list, other_val)
    save_checkpoint('Covid_19_CT_Images_Random'+str(epoch)+'.pt', model_Ran, optimizer, result, epoch)
    epoch_save=epoch

Train Epoch: 0 [0%]	Loss: 0.717220
epoch 0 training loss: 0.4377221640897915 acc: 0.8115727002967359


/var/folders/1q/x916rngd0bl3jwtyp7zz6h1w0000gn/T/ipykernel_830/3076453557.py:12: RuntimeWarning: invalid value encountered in scalar divide
  sens[n]=TP/(TP+FN)
/var/folders/1q/x916rngd0bl3jwtyp7zz6h1w0000gn/T/ipykernel_830/3076453557.py:13: RuntimeWarning: invalid value encountered in scalar divide
  prec[n]=TP/(TP+FP)


epoch 0 validation acc: 0.78333336
saved: Covid_19_CT_Images_Random0.pt
Train Epoch: 1 [0%]	Loss: 0.313438
epoch 1 training loss: 0.2304337655659765 acc: 0.9070227497527201
epoch 1 validation acc: 0.8333333
saved: Covid_19_CT_Images_Random1.pt
Train Epoch: 2 [0%]	Loss: 0.167972
epoch 2 training loss: 0.1785069305333309 acc: 0.9213649851632048
epoch 2 validation acc: 0.53333336
saved: Covid_19_CT_Images_Random2.pt
Train Epoch: 3 [0%]	Loss: 0.072565
epoch 3 training loss: 0.1279698265134357 acc: 0.9515331355093967
epoch 3 validation acc: 0.6
saved: Covid_19_CT_Images_Random3.pt
Train Epoch: 4 [0%]	Loss: 0.051702
epoch 4 training loss: 0.07191089003663365 acc: 0.9723046488625123
epoch 4 validation acc: 0.9
saved: Covid_19_CT_Images_Random4.pt
Train Epoch: 5 [0%]	Loss: 0.010340
epoch 5 training loss: 0.03170907261301181 acc: 0.9886251236399605
epoch 5 validation acc: 0.85
saved: Covid_19_CT_Images_Random5.pt


In [14]:
acc_test, other_test = test(model_Ran, device, loader_test)
print('Test acc:', acc_test)

Test acc: 0.76


/var/folders/1q/x916rngd0bl3jwtyp7zz6h1w0000gn/T/ipykernel_830/3076453557.py:12: RuntimeWarning: invalid value encountered in scalar divide
  sens[n]=TP/(TP+FN)
/var/folders/1q/x916rngd0bl3jwtyp7zz6h1w0000gn/T/ipykernel_830/3076453557.py:13: RuntimeWarning: invalid value encountered in scalar divide
  prec[n]=TP/(TP+FP)


In [15]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_Pre = PretrainedResNet18()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adamax(model_Pre.parameters(), lr=0.001, weight_decay=1e-4)

/Users/yemiakj10/Anaconda/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/yemiakj10/Anaconda/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [16]:
loader_train, loader_val, loader_test = get_dataloader()

loss_train_list=[]
acc_train_list=[]
acc_val_list=[]
epoch_save=-1

In [17]:
for epoch in range(epoch_save+1, 6): #change 100 to a larger number if necessary
    #-------- training --------------------------------
    loss_train, acc_train =train(model_Pre, device, optimizer, loader_train, epoch)    
    loss_train_list.append(loss_train)
    acc_train_list.append(acc_train)
    print('epoch', epoch, 'training loss:', loss_train, 'acc:', acc_train)
    #-------- validation --------------------------------
    acc_val, other_val = test(model_Pre, device, loader_val)
    acc_val_list.append(acc_val)
    print('epoch', epoch, 'validation acc:', acc_val)
    #-------- test --------------------------------------
    #acc_test, other_test = test(model_Pre, device, loader_test)
    #acc_test_list.append(acc_test)
    #print('epoch', epoch, 'test acc:', acc_test)
    #--------save model-------------------------
    result = (loss_train_list, acc_train_list, 
              acc_val_list, other_val)
    save_checkpoint('Covid_19_CT_Images_Pretrained'+str(epoch)+'.pt', model_Pre, optimizer, result, epoch)
    epoch_save=epoch

Train Epoch: 0 [0%]	Loss: 0.763352
epoch 0 training loss: 0.29221851969487034 acc: 0.8882294757665677


/var/folders/1q/x916rngd0bl3jwtyp7zz6h1w0000gn/T/ipykernel_830/3076453557.py:12: RuntimeWarning: invalid value encountered in scalar divide
  sens[n]=TP/(TP+FN)
/var/folders/1q/x916rngd0bl3jwtyp7zz6h1w0000gn/T/ipykernel_830/3076453557.py:13: RuntimeWarning: invalid value encountered in scalar divide
  prec[n]=TP/(TP+FP)


epoch 0 validation acc: 0.76666665
saved: Covid_19_CT_Images_Pretrained0.pt
Train Epoch: 1 [0%]	Loss: 0.056118
epoch 1 training loss: 0.05128537997370586 acc: 0.983679525222552
epoch 1 validation acc: 0.96666664
saved: Covid_19_CT_Images_Pretrained1.pt
Train Epoch: 2 [0%]	Loss: 0.004243
epoch 2 training loss: 0.03969997144758963 acc: 0.9846686449060337
epoch 2 validation acc: 0.96666664
saved: Covid_19_CT_Images_Pretrained2.pt
Train Epoch: 3 [0%]	Loss: 0.036026
epoch 3 training loss: 0.022294436599622713 acc: 0.9930761622156281
epoch 3 validation acc: 0.9166667
saved: Covid_19_CT_Images_Pretrained3.pt
Train Epoch: 4 [0%]	Loss: 0.001679
epoch 4 training loss: 0.015381679459551378 acc: 0.9960435212660732
epoch 4 validation acc: 0.8333333
saved: Covid_19_CT_Images_Pretrained4.pt
Train Epoch: 5 [0%]	Loss: 0.002155
epoch 5 training loss: 0.00672489255543951 acc: 0.9990108803165183
epoch 5 validation acc: 0.95
saved: Covid_19_CT_Images_Pretrained5.pt


In [18]:
acc_test, other_test = test(model_Pre, device, loader_test)
print('Test acc:', acc_test)

Test acc: 0.98


/var/folders/1q/x916rngd0bl3jwtyp7zz6h1w0000gn/T/ipykernel_830/3076453557.py:12: RuntimeWarning: invalid value encountered in scalar divide
  sens[n]=TP/(TP+FN)
/var/folders/1q/x916rngd0bl3jwtyp7zz6h1w0000gn/T/ipykernel_830/3076453557.py:13: RuntimeWarning: invalid value encountered in scalar divide
  prec[n]=TP/(TP+FP)


In [19]:
print("Model's state_dict:")
for param_tensor in model_Ran.state_dict():
    print(param_tensor, "\t", model_Ran.state_dict()[param_tensor].size())

Model's state_dict:
resnet.conv1.weight 	 torch.Size([64, 3, 7, 7])
resnet.bn1.weight 	 torch.Size([64])
resnet.bn1.bias 	 torch.Size([64])
resnet.bn1.running_mean 	 torch.Size([64])
resnet.bn1.running_var 	 torch.Size([64])
resnet.bn1.num_batches_tracked 	 torch.Size([])
resnet.layer1.0.conv1.weight 	 torch.Size([64, 64, 3, 3])
resnet.layer1.0.bn1.weight 	 torch.Size([64])
resnet.layer1.0.bn1.bias 	 torch.Size([64])
resnet.layer1.0.bn1.running_mean 	 torch.Size([64])
resnet.layer1.0.bn1.running_var 	 torch.Size([64])
resnet.layer1.0.bn1.num_batches_tracked 	 torch.Size([])
resnet.layer1.0.conv2.weight 	 torch.Size([64, 64, 3, 3])
resnet.layer1.0.bn2.weight 	 torch.Size([64])
resnet.layer1.0.bn2.bias 	 torch.Size([64])
resnet.layer1.0.bn2.running_mean 	 torch.Size([64])
resnet.layer1.0.bn2.running_var 	 torch.Size([64])
resnet.layer1.0.bn2.num_batches_tracked 	 torch.Size([])
resnet.layer1.1.conv1.weight 	 torch.Size([64, 64, 3, 3])
resnet.layer1.1.bn1.weight 	 torch.Size([64])
resnet.

In [20]:
print("Model's state_dict:")
for param_tensor in model_Pre.state_dict():
    print(param_tensor, "\t", model_Pre.state_dict()[param_tensor].size())

Model's state_dict:
features.0.weight 	 torch.Size([64, 3, 7, 7])
features.1.weight 	 torch.Size([64])
features.1.bias 	 torch.Size([64])
features.1.running_mean 	 torch.Size([64])
features.1.running_var 	 torch.Size([64])
features.1.num_batches_tracked 	 torch.Size([])
features.4.0.conv1.weight 	 torch.Size([64, 64, 3, 3])
features.4.0.bn1.weight 	 torch.Size([64])
features.4.0.bn1.bias 	 torch.Size([64])
features.4.0.bn1.running_mean 	 torch.Size([64])
features.4.0.bn1.running_var 	 torch.Size([64])
features.4.0.bn1.num_batches_tracked 	 torch.Size([])
features.4.0.conv2.weight 	 torch.Size([64, 64, 3, 3])
features.4.0.bn2.weight 	 torch.Size([64])
features.4.0.bn2.bias 	 torch.Size([64])
features.4.0.bn2.running_mean 	 torch.Size([64])
features.4.0.bn2.running_var 	 torch.Size([64])
features.4.0.bn2.num_batches_tracked 	 torch.Size([])
features.4.1.conv1.weight 	 torch.Size([64, 64, 3, 3])
features.4.1.bn1.weight 	 torch.Size([64])
features.4.1.bn1.bias 	 torch.Size([64])
features.4

In [21]:
# Specify the directory where you want to save the model
directory = "/Users/yemiakj10/Desktop/MSDS 2/Machine Learning/Main Story/GradCam"

# Specify the file name for the model
filename = "model_Ran.pth"

# Combine directory and file name to create the complete file path
PATH = os.path.join(directory, filename)

In [22]:
torch.save(model_Ran.state_dict(), PATH)

In [23]:
# Specify the directory where you want to save the model
directory = "/Users/yemiakj10/Desktop/MSDS 2/Machine Learning/Main Story/GradCam"

# Specify the file name for the model
filename = "model_Ran.pth"

# Combine directory and file name to create the complete file path
PATH = os.path.join(directory, filename)

In [ ]:
# Specify the directory where you want to save the model
directory = "/Users/yemiakj10/Desktop/MSDS 2/Machine Learning/Main Story/GradCam"

# Specify the file name for the model
filename = "model_Pre.pth"

# Combine directory and file name to create the complete file path
PATH = os.path.join(directory, filename)

In [24]:
torch.save(model_Pre.state_dict(), PATH)